# Imports

Make sure you have the packages installed. You might need to pip install some of them, the console should tell you if you are missing any packages when you try to run the code.

In [ ]:
# standard imports
import random
import math
import numpy as np
import matplotlib.pyplot as plt

# allows flexible tensor manipulation
import einops

# progress bars during training and sampling
from tqdm.auto import tqdm

# PyTorch for general machine learning
import torch
import torch.nn as nn
from torch.optim import Adam

# help in displaying images
from torchvision.utils import make_grid
from torchvision.transforms.functional import to_pil_image

# libraries for data and dataset processing
from torch.utils.data import DataLoader
from torchvision.transforms import Compose, ToTensor, Lambda
from torchvision.datasets.mnist import FashionMNIST

# libraries for quantative metrics calculations
from torchvision.utils import save_image
from torch_fidelity import calculate_metrics
import os

# Helper function to display images

In [ ]:
def display_images(images, labels=None, title="", num_samples=20, cols=4):
    # ensure we don't exceed the number of available images
    images = images[:min(num_samples, len(images))]

    # create a grid of images
    # normalize each image
    grid = make_grid(images, nrow=cols, normalize=True, scale_each=True)  # Adjust grid columns
    
    # convert the grid to a PIL image
    grid_img = to_pil_image(grid)
    
    # plot
    plt.figure(figsize=(12, 12))  # You can adjust the figure size as needed
    plt.imshow(grid_img, cmap="gray")
    plt.title(title, fontsize=20)
    plt.axis('off')
    
    # if labels are provided, display them (note: labels arent displayed very well)
    if labels is not None:
        num_images = len(images)
        rows = (num_images + cols - 1) // cols  # Calculate the number of rows in the grid
        for i, label in enumerate(labels[:num_images]):
            plt.text(
                (i % cols) * grid_img.width / cols, 
                (i // cols + 1) * grid_img.height / rows - 10,  # Adjust text position
                label, 
                horizontalalignment='center',
                fontsize=10,
                color='white',
                weight='bold'
            )
    plt.show()

# Pre-process data

In [ ]:
# transform to tensor and normalize [-1,1]
transform = Compose([
    ToTensor(),
    Lambda(lambda x: (x - 0.5) * 2)
])

batch_size = 128

# load FashionMNIST dataset
train_dataset = FashionMNIST("./data", download=True, train=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

# Classifier-Free Guidance DDPM Implmentation

In [ ]:
# get the device (GPU if available)
# this is essential as we want to train on our GPU!
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# print the device being used to make sure we have access
print(f"Using device: {device}\t" + (f"{torch.cuda.get_device_name(0)}" if torch.cuda.is_available() else "CPU"))

In [ ]:
# variance scheduler class to pre-compute noise values
class VarianceScheduler:
    def __init__(self, beta1, beta2, T, device, schedule_type="linear", s=0.008, beta_max=0.999):
        # make sure beta values are in defined bounds
        assert 0 < beta1 < beta2 < 1.0, "beta1 and beta2 must be in (0, 1)"
        self.device = device
        self.schedule_type = schedule_type
        self.beta1 = beta1 # start beta
        self.beta2 = beta2 # end beta
        self.T = T # total timesteps in diffusion process
        self.s = s # smoothing constant for cosine schedule
        self.beta_max = beta_max # capped beta value for cosine schedule
        self.schedule = self.compute_schedule()

    def compute_schedule(self):
        timesteps = torch.arange(0, self.T, dtype=torch.float32).to(self.device) # define timesteps 0 to T and move to gpu
        if self.schedule_type == 'linear':
            #compute linear schedule from Ho et al.'s DDPM paper
            betas = torch.linspace(self.beta1, self.beta2, self.T).to(self.device) 
        elif self.schedule_type == 'cosine':
            #compute cosine schedule using equations from Nichol et al.'s Improved DDPM paper
            t_scaled = (timesteps / self.T + self.s) / (1 + self.s) * (torch.pi / 2)
            f_t = torch.cos(t_scaled).pow(2)
            f_0 = torch.cos(torch.tensor(self.s / (1 + self.s) * (torch.pi / 2))).pow(2)
            alpha_bars = f_t / f_0
            alpha_bars_prev = torch.cat([torch.tensor([1]).to(self.device), alpha_bars[:-1]])
            betas = 1 - alpha_bars / alpha_bars_prev
            betas = torch.clip(betas, 0, self.beta_max)
        else:
            raise ValueError("Unknown schedule type: {}".format(self.schedule_type)) # error for unknown schedule type

        # calculate alphas and alpha_bars as shown in DDPM paper (reparameterization trick)
        alphas = 1 - betas
        alpha_bars = torch.tensor([torch.prod(alphas[:i + 1]) for i in range(len(alphas))]).to(self.device)

        # return dictionary of betas, alphas, and alpha_bars
        schedule = {
            "betas": betas,
            "alphas": alphas,
            "alpha_bars": alpha_bars,
        }
        return schedule

In [ ]:
# DDPM class
class CFG_DDPM(nn.Module):
    def __init__(self, network, T=1000, beta1=10 ** -4, beta2=0.02, schedule_type='linear', device=None):
        super(CFG_DDPM, self).__init__()
        self.device = device
        self.network = network.to(device) # move the unet to the gpu
        self.T = T # timesteps T
        self.variance_scheduler = VarianceScheduler(beta1, beta2, T, device, schedule_type) # Initialize variance schdeuler

        # use register_buffer to store each schedule component
        for k, v in self.variance_scheduler.schedule.items():
            self.register_buffer(k, v)

    # noising function (forward diffusion)
    def forward(self, x0, t, noise=None):
        # Make input image more noisy (we can directly skip to the desired step)
        n, channel, height, width = x0.shape
        a_bar = self.alpha_bars[t]

        #if no noise is passed then calculate a random noise mask of the same dimensions as the image
        if noise is None:
            noise = torch.randn(n, channel, height, width).to(self.device) # move to gpu

        # noising function
        noisy = a_bar.sqrt().reshape(n, 1, 1, 1) * x0 + (1 - a_bar).sqrt().reshape(n, 1, 1, 1) * noise
        # return noised image
        return noisy

    # added label conditioning for CFG
    def backward(self, x, t, labels):
        return self.network(x, t, labels)  # switch unet to eval mode (stops training behaviors)

    # MODIFIED SAMPLING!
    # produces n_samples of specified class labels in list labels. Note that the nuimber of provided labels needs to = n_samples
    def sample_images(self, labels, n_samples=16, cfg_scale=3.0):
        self.network.eval()  # set unet to eval mode

        # torch.no_grad() turns off gradient computation (not needed when sampling) for reduced memory usage and faster computations
        with torch.no_grad():
            labels = torch.tensor(labels, dtype=torch.long, device=self.device)
            x = torch.randn(n_samples, *(1,28,28)).to(self.device)

            # loop backward through entire diffusion process (T-1 -> 0)
            for i in tqdm(reversed(range(1, self.T)), position=0):
                # create tensor filled with the current timestep, shaped for each sampl
                t = (torch.ones(n_samples) * i).long().to(self.device)
                
                # noise prediction conditioned on class labels
                conditional_predicted_noise = self.backward(x, t, labels)
                
                if cfg_scale > 0:
                    # unconditional prediction
                    unconditional_predicted_noise = self.backward(x, t, None)
                    # linear interpolation between conditional and unconditional noise based on cfg_scale
                    predicted_noise = torch.lerp(unconditional_predicted_noise, conditional_predicted_noise, cfg_scale)
                
                # get alpha, alpha_bar, and beta values for the current timestep from the precomputed schedule
                alpha_t = self.alphas[i]
                alpha_t_bar = self.alpha_bars[i]
                beta_t = self.betas[i]
                
                # determine noise to be added at this timestep
                if i > 1:
                    # sample random noise if not at the final timestep
                    noise = torch.randn_like(x)
                else:
                    # use no noise at the final timestep to avoid adding unnecessary randomness
                    noise = torch.zeros_like(x)
                
                # update image tensor using reverse diffusion formula
                x = 1 / torch.sqrt(alpha_t) * (x - ((1 - alpha_t) / torch.sqrt(1 - alpha_t_bar)) * predicted_noise) + torch.sqrt(beta_t) * noise

        # reset unet to training mode
        self.network.train()  # Set the model back to train mode
        return x

# Conditional U-Net
We now build upon the U-Net that we used in our unconditional DDPM implementation \
We still acknowledge the GitHub implementation which our unconditional model was heavily inspired by, as it was a foundational step in coding the conditional diffusion model. (Cited in dissertation report).

In [ ]:
# U-Net helpers

def sinusoidal_embedding(n, dim):
    # returns the standard positional embedding
    if not isinstance(n, int) or not isinstance(dim, int) or n < 1 or dim < 1:
        raise ValueError("both 'n' and 'dim' must be positive integers!")
    
    # vectorized calculation of the frequency terms
    wk = torch.pow(10_000, -torch.arange(0, dim, 2, dtype=torch.float32) / dim)
    wk = wk.reshape(1, -1)
    # calculate positional information
    t = torch.arange(n, dtype=torch.float32).reshape(n, 1)
    
    # create the embedding matrix
    embedding = torch.zeros(n, dim)
    embedding[:, 0::2] = torch.sin(t * wk)
    embedding[:, 1::2] = torch.cos(t * wk)

    return embedding
 
# embedding and label embedding are concatenated before being run through the MLP
def make_context_embedding(embedding_dim, out_features):
    """Utility function to create time embedding layers."""
    return nn.Sequential(
        nn.Linear(embedding_dim, out_features),
        nn.SiLU(),
        nn.Linear(out_features, out_features)
    )


class ConvBlock(nn.Module):
    def __init__(self, shape, in_channels, out_channels, kernel=3, stride=1, padding=1, activation=None, normalize=True):
        super(ConvBlock, self).__init__()
        self.layernorm = nn.LayerNorm(shape)
        self.convolution1 = nn.Conv2d(in_channels, out_channels, kernel, stride, padding)
        self.convolution2 = nn.Conv2d(out_channels, out_channels, kernel, stride, padding)
        self.activation = nn.SiLU() if activation is None else activation
        self.normalize = normalize

    def forward(self, x):
        out = self.layernorm(x) if self.normalize else x
        out = self.convolution1(out)
        out = self.activation(out)
        out = self.convolution2(out)
        out = self.activation(out)
        return out

In [ ]:
class UNet(nn.Module):
    def __init__(self, T=1000, embedding_dim=100, num_classes=10):
        super(UNet, self).__init__()

        # initialize our sinusoidal time embedding
        self.time_embed = nn.Embedding(T, embedding_dim)
        self.time_embed.weight.data = sinusoidal_embedding(T, embedding_dim)
        self.time_embed.requires_grad_(False)

        # make context embeddings for every layer
        self.context_embedding1 = make_context_embedding(embedding_dim, 1)
        self.context_embedding2 = make_context_embedding(embedding_dim, 10)
        self.context_embedding3 = make_context_embedding(embedding_dim, 20)
        self.context_embedding_mid = make_context_embedding(embedding_dim, 40)
        self.context_embedding4 = make_context_embedding(embedding_dim, 80)
        self.context_embedding5 = make_context_embedding(embedding_dim, 40)
        self.context_embedding_out = make_context_embedding(embedding_dim, 20)

        # create a class label embedding
        if num_classes is not None:
            self.label_emb = nn.Embedding(num_classes, embedding_dim)

        # first residual convolutional block, made up of 3 convolutional blocks
        # increases our feature depth from 1 to 10
        self.residual1 = nn.Sequential(
            ConvBlock((1, 28, 28), 1, 10),
            ConvBlock((10, 28, 28), 10, 10),
            ConvBlock((10, 28, 28), 10, 10)
        )
        
        # first downsampling, decreases image dimensions to (14x14)
        self.downsample1 = nn.Conv2d(10, 10, 4, 2, 1)

        # second residual convolutional block, increases feature depth to 20
        self.residual2 = nn.Sequential(
            ConvBlock((10, 14, 14), 10, 20),
            ConvBlock((20, 14, 14), 20, 20),
            ConvBlock((20, 14, 14), 20, 20)
        )

        # second downsampling, decreases image dimensions to 7x7
        self.downsample2 = nn.Conv2d(20, 20, 4, 2, 1)

        # third residual convolutional block, increases feature depth to 40
        self.residual3 = nn.Sequential(
            ConvBlock((20, 7, 7), 20, 40),
            ConvBlock((40, 7, 7), 40, 40),
            ConvBlock((40, 7, 7), 40, 40)
        )

        # third downsampling decreases spatial dimensions to 3x3
        self.downsample3 = nn.Sequential(
            nn.Conv2d(40, 40, 2, 1),
            nn.SiLU(),
            nn.Conv2d(40, 40, 4, 2, 1)
        )

        # bottleneck, decreases feature depth to 20 and then increases back to 40
        self.bottleneck = nn.Sequential(
            ConvBlock((40, 3, 3), 40, 20),
            ConvBlock((20, 3, 3), 20, 20),
            ConvBlock((20, 3, 3), 20, 40)
        )

        # first upsampling, increase the spatial dimensions to 7x7
        self.upsample1 = nn.Sequential(
            nn.ConvTranspose2d(40, 40, 4, 2, 1),
            nn.SiLU(),
            nn.ConvTranspose2d(40, 40, 2, 1)
        )

        # fourth residual convolutional block, decreases feature depth from 80 to 20
        # the feature depth is 80 as we concatenate the correponding downsampling layers data to this layer
        # via skip connection
        self.residual4 = nn.Sequential(
            ConvBlock((80, 7, 7), 80, 40),
            ConvBlock((40, 7, 7), 40, 20),
            ConvBlock((20, 7, 7), 20, 20)
        )

        # second upsampling, increase the spatial dimensions to 14x14
        self.upsample2 = nn.ConvTranspose2d(20, 20, 4, 2, 1)

        # fifth residual convolutional block, decreases feature depth from 40 to 20
        # the feature depth 40 due to concatenation from downsampling stage
        self.residual5 = nn.Sequential(
            ConvBlock((40, 14, 14), 40, 20),
            ConvBlock((20, 14, 14), 20, 10),
            ConvBlock((10, 14, 14), 10, 10)
        )

        # third upsampling, increase spatial dimensions to 29x29
        self.upsample3 = nn.ConvTranspose2d(10, 10, 4, 2, 1)
        self.residual_out = nn.Sequential(
            ConvBlock((20, 28, 28), 20, 10),
            ConvBlock((10, 28, 28), 10, 10),
            ConvBlock((10, 28, 28), 10, 10, normalize=False)
        )

        # final convolution for (10,28,28) -> (1,28,28)
        self.final_convolution = nn.Conv2d(10, 1, 3, 1, 1)

    def forward(self, x, t, labels):
        time_embedding = self.time_embed(t).squeeze(1)
        # print(f"Adjusted Time embedding shape: {time_embedding.shape}")

        # if a label is passed create a context embedding of both time and label
        if labels is not None:
            label_embedding = self.label_emb(labels)
            # print(f"Time embedding shape: {time_embedding.shape}")
            # print(f"Label embedding shape: {label_embedding.shape}")
            context_embedding = time_embedding + label_embedding
        else:
            # if there is no label then we just condition on temporal information
            context_embedding = time_embedding

        n = len(x)
        out1 = self.residual1(x + self.context_embedding1(context_embedding).reshape(n, -1, 1, 1))
        out2 = self.residual2(self.downsample1(out1) + self.context_embedding2(context_embedding).reshape(n, -1, 1, 1)) 
        out3 = self.residual3(self.downsample2(out2) + self.context_embedding3(context_embedding).reshape(n, -1, 1, 1))

        out_mid = self.bottleneck(self.downsample3(out3) + self.context_embedding_mid(context_embedding).reshape(n, -1, 1, 1))

        # skip connection
        out4 = torch.cat((out3, self.upsample1(out_mid)), dim=1)  # (N, 80, 7, 7) 
        out4 = self.residual4(out4 + self.context_embedding4(context_embedding).reshape(n, -1, 1, 1))

        # skip connection
        out5 = torch.cat((out2, self.upsample2(out4)), dim=1)  # (N, 40, 14, 14)
        out5 = self.residual5(out5 + self.context_embedding5(context_embedding).reshape(n, -1, 1, 1))

        # skip connection
        out = torch.cat((out1, self.upsample3(out5)), dim=1)  # (N, 20, 28, 28)
        out = self.residual_out(out + self.context_embedding_out(context_embedding).reshape(n, -1, 1, 1))

        out = self.final_convolution(out)

        return out

# Training loop

In [ ]:
def training_loop(ddpm, dataloader, n_epochs, optim, device, sample_images=True, store="ddpm.pt"):
    T = ddpm.T
    mse = nn.MSELoss() #loss
    best_loss = float("inf") # we initialize a best loss tracker
    losses = [] # a store for losses to plot

    for epoch in tqdm(range(n_epochs), desc="Training progress"):
        epoch_loss = 0.0
        for step, (images, labels) in enumerate(tqdm(dataloader, leave=False, desc=f"Epoch {epoch + 1}/{n_epochs}")):
            # load data and move to device
            x0 = images.to(device)
            labels = labels.to(device)
            n = len(x0)

            # pick some noise for each of the images in the batch, a timestep
            eta = torch.randn_like(x0).to(device)
            t = torch.randint(0, T, (n,)).to(device)

            # computing the noisy image based on x0 and the time-step (forward process)
            noisy_imgs = ddpm(x0, t, eta)

            # 10% of the time, we should drop conditioning on labels
            if np.random.random() < 0.1:
                labels = None
                
            # getting model estimation of noise based on the images, timesteps, and labels
            eta_theta = ddpm.backward(noisy_imgs, t.reshape(n, -1), labels)

            # optimize the MSE between the actual noise and the predicted noise
            loss = mse(eta_theta, eta)
            optim.zero_grad()
            loss.backward()
            optim.step()
            # calculates average loss over epoch
            epoch_loss += loss.item() * len(x0) / len(dataloader.dataset)
        # store epoch loss for plotting
        losses.append(epoch_loss)
        
        # if sample_images then, display images generated at this epoch
        if display:
            # create a list of labels from 0 to 9 for targeted sampling
            desired_labels = list(range(10))
            # use the sample_images method from the DDPM class
            generated_images = ddpm.sample_images(labels=desired_labels, n_samples=10)
            # display the generated images
            display_images(generated_images, labels=None, title=f"Images generated at epoch {epoch + 1}", num_samples=10, cols=10)
        
        log_string = f"Loss at epoch {epoch + 1}: {epoch_loss:.3f}"

        # store the model if we attain  a new low loss
        if best_loss > epoch_loss:
            best_loss = epoch_loss
            torch.save(ddpm.state_dict(), store)
            log_string += " model stored"

        print(log_string)  # print epoch loss and whether model was stored
        
    # Plotting the loss
    plt.figure(figsize=(10, 5))
    plt.plot(range(1, n_epochs + 1), losses, marker='o', linestyle='-')
    plt.title("Training Loss Per Epoch")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.grid(True)
    plt.show()

In [ ]:
# defining model
# values defined as originally suggested by the ddpm paper authors
T = 1000
beta1 = 10 ** -4
beta2 = 0.02
schedule_type = "linear"
num_classes = 10 # 10 classes in FashionMNIST

# initialize ddpm for training
ddpm = CFG_DDPM(UNet(T, num_classes=num_classes), T=T, beta1=beta1, beta2=beta2, schedule_type=schedule_type, device=device)

In [ ]:
# Training
# we save the best loss model and the final 300 epoch trained model
store = "best_loss_ddpm_cfg.pt"
epochs = 300
lr = 0.001

training_loop(ddpm, train_loader, epochs, optim=Adam(ddpm.parameters(), lr), device=device, sample_images=True, store=store)
torch.save(ddpm.state_dict(), 'ddpm_cfg.pt')

# Targeted sampling

In [ ]:
# Optionally load a pre-trained model here.
# T = 1000
# beta1 = 10 ** -4
# beta2 = 0.02
# schedule_type = "linear"
# num_classes = 10
# model = CFG_DDPM(UNet(T, num_classes=num_classes), T=T, beta1=beta1, beta2=beta2, device=device, schedule_type=schedule_type)
# model.load_state_dict(torch.load(model path))

desired_labels = []
n_samples = 10
for i in range (10):
    desired_labels += ([i]*10)

generated_images = ddpm.sample_images(labels=desired_labels, n_samples=100)
display_images(generated_images, labels=None, title="Unconditional Diffusion Generations", num_samples=100, cols=10)

# Quantitative Metrics Calculation (FID)

In [ ]:
def save_images(images, folder="saved_images"):
    # takes a tensor of images and saves them in a specified folder
    if not isinstance(images, torch.Tensor):
        raise ValueError("Images should be a PyTorch tensor")

    images = images.detach().cpu()

    # normalize and prepare images
    os.makedirs(folder, exist_ok=True)
    for i, img_tensor in enumerate(images):
        img = img_tensor.squeeze()  # remove color channels
        img_np = img.numpy()
        plt.imsave(os.path.join(folder, f'image_{i}.png'), img_np, cmap='gray')

def compute_fid(real_images_path, fake_images_path):
    # computes the FID score between two sets of images located at the given folder paths.
    # we set isc and kid to false as we only want to calculate FID scores
    # we set samples_find_deep=True as we need to find images recursively within our folders
    metrics = calculate_metrics(input1=real_images_path, input2=fake_images_path, cuda=True, isc=False, fid=True, kid=False, samples_find_deep=True)
    return metrics['frechet_inception_distance']

In [ ]:
# Optionally load a pre-trained model here.
# T = 1000
# beta1 = 10 ** -4
# beta2 = 0.02
# schedule_type = "linear"
# num_classes = 10
# model = CFG_DDPM(UNet(T, num_classes=num_classes), T=T, beta1=beta1, beta2=beta2, device=device, schedule_type=schedule_type)
# model.load_state_dict(torch.load(model path))

# number of loops and samples
n_loops = 5
n_samples_per_class = 1000
n_classes = 10
total_samples = n_samples_per_class * n_classes

# base folder to store generated images
base_folder_name = "generated_images_conditional"

for loop in range(n_loops):
    # generate labels for each class
    labels = []
    for class_label in range(n_classes):
        labels += [class_label] * n_samples_per_class

    # sample images using the generated labels
    generated_images = ddpm.sample_images(labels=labels, n_samples=total_samples)

    # folder to save images in a loop-specific subfolder
    loop_folder = os.path.join(base_folder_name, f'set_{loop + 1}')
    if not os.path.exists(loop_folder):
        os.makedirs(loop_folder)
    
    # save the images in the specified loop folder
    save_images(generated_images, loop_folder)

In [ ]:
# load FashionMNIST dataset
dataset = FashionMNIST(root="../data", train=True, transform=ToTensor(), download=True)

# initialize a list to hold selected images
n_images_per_class = 10000
selected_images = []

for class_id in range(10):
    # filter indices for the current class
    class_indices = [i for i, (_, label) in enumerate(dataset) if label == class_id]
    # Randomly select 5000 indices for the current class without replacement
    selected_indices = np.random.choice(class_indices, n_images_per_class, replace=False)
    
    # append selected images to the list
    for idx in selected_indices:
        image, _ = dataset[idx]
        selected_images.append(image)

# convert list of selected images to a tensor
real_images = torch.stack(selected_images)

# save the selected images
save_images(real_images, "real_images_conditional")

In [ ]:
# compute and print FID score of synthetic vs authentic images
fid_score = compute_fid("real_images_conditional", "generated_images_conditional")
print(f"FID Score: {fid_score}")